In [ ]:
###Load necessary modules###
import keras as k
import numpy as np
import tensorflow as tf
import matplotlib.image as mpimg 
import cv2

In [ ]:
###Load segmentation model###
seg_mod = tf.keras.models.load_model(" ") #include path to Soy_Segmentation.hdf5 within the quotation marks

In [ ]:
###Function to break image down into neightborhood format###
def img_to_mat_nbhd(image):
    k=0
    r,c = np.shape(image)[0],np.shape(image)[1]
    img_pad = np.zeros((r+1,c+1,3))
    img_pad[1:(r+1),1:(c+1),:] = image[:,:,0:3]
    img_mat = np.zeros((r*c,27))
    ind_r = np.delete(np.add(list(range(r)),1),r-1)
    ind_c = np.delete(np.add(list(range(c)),1),c-1)
    for i in ind_r:
        for j in ind_c:
            img_mat[(i*c+j-1),:] = np.array(np.concatenate((np.ndarray.flatten(np.transpose(img_pad[(i-1):(i+2),(j-1):(j+2),0])),
                                           np.ndarray.flatten(np.transpose(img_pad[(i-1):(i+2),(j-1):(j+2),1])),
                                           np.ndarray.flatten(np.transpose(img_pad[(i-1):(i+2),(j-1):(j+2),2])))))
            k+=1
    return(img_mat)

In [ ]:
###Segment image using pretrained segmentation model###
img = mpimg.imread(" ") #Include path to example image within quotation marks
i_mat = img_to_mat_nbhd(img)
prob_vec = seg_mod.predict_proba(i_mat)
prob_lab = np.zeros(len(prob_vec))
for i in range(len(prob_lab)):
    if prob_vec[i]>0.995:
        prob_lab[i] = 1
r,c = np.shape(img)[0], np.shape(img)[1]
bin_img = np.zeros((r,c))
for i in range(r):
    bin_img[i,:] = prob_lab[(i*c):((i+1)*c)]
    res = np.multiply(bin_img,255)
cv2.imwrite(" ",res) #Include path to which binary image will be written within quotation marks

In [ ]:
###Opening operation on binary image using 5x5 diamond-shaped kernel###
diam_ker_5 = np.array([[0,0,1,0,0],[0,1,1,1,0],[1,1,1,1,1],[0,1,1,1,0],[0,0,1,0,0]],np.uint8)
cv2.imread(" ") #Include path to binary image within quotation marks
img_open = cv2.morphologyEx(img,cv2.MORPH_OPEN,diam_ker_5)
cv2.imwrite(" ",img_open) #Include path to which opened binary image will be written within quotation marks

In [ ]:
###Obtain height, width and size measurements from opened binary image###
row_sums = np.sum(img_open,axis=1)
pixel_ht = np.subtract(860,np.min(np.where(row_sums>1))) #860 is the pixel position of the top of the plant's pot

col_sums = np.sum(img.open,axis=0)
pixel_wdt = np.subtract(np.max(np.where(col_sums>1)),np.min(np.where(col_sums>1)))

pixel_size = np.sum(img.open)